# Assignment 10: X-Ray Pneumonia Detection (M.Sc Students)

==> *Write*
* *Tuo ZHANG* 
* *matr. nr.* 
* *Computational Linguistics*
* M.Sc.

*of all assignment group participants here. (double klick here to edit)*

In this assignment, you are tasked with developing your own classifier for pneumonia in X-ray images. You will go through to the complete ML development cycle from loading and preprocessing your data to evaluating your models.

Extract the X-Ray dataset https://rssiste.sharepoint.com/:u:/s/analyticcomputing/ERMTk8Wm091Crev9tV0NEBUBv7ue3bRSG8iiftCCjbOKhA?e=UcjaFB to the same directory as your Jupyter notebook. The data is already split into a training, validation, and testing set. The dataset originates from the paper [Identifying Medical Diagnoses and Treatable Diseases by Image-Based Deep Learning
](https://www.cell.com/cell/fulltext/S0092-8674(18)30154-5).

You may use any packages, we encountered during the exercises (numpy, matplotlib, scikit-learn, scikit-image, pandas, pytorch) as well as the Python standard library.

You should (at least) address the following points in your development process:

- The dataset is imbalanced. Do at least one of the following:
    - Augment your dataset by including rotated, flipped, or brightened images. This will also improve the generalization capabilities of your model.
    - or: Modify your objective function by weighting the classes differently.
- Optimize the hyperparameters of your models using grid-search or random-search on the validation set.
- Consider at least two classes of models, e.g. CNN and SVM. At least one of your model classes should be some type of neural network implemented in PyTorch.
- After the hyperparameter optimization, select the best-performing models of each class. Evaluate these models on the testing data and visualize your results.

Points (200):

    1. Model Definitions and Training : 80
    2. Model Evaluations : 80
    3. Data Augmentation and Hyperparameter Searching : 40

*Note*! 

This assignment scores (200) counts in 60% threshold (from all 10 assignment, including this project) to be passed for exam. If you already have enough scores then you are not obliged to complete this project (i.e. you already have more than 60% of ALL scores from previous assignment, again, including this project). Otherwise you should do this assignments that it provide double of usual points (200); thus it can increase the overall submisson score. 

For example, if you have more than 660 points out of 1100 total (900-previous assignment + 200-this assignment), then you do not have to perform the assignment . You can consult all of your accumulated points in Illias.


You have two weeks to complete this assignment.

In [1]:
import numpy as np
import torch  # Package name: torch (for pip), pytorch (for conda)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import cv2


def load_dataset():
    import os

    def load_data(directory):
        directories = (d for d in os.listdir(directory)
                       if os.path.isdir(os.path.join(directory, d)))
        labels = []
        images = []
        count_1 = 0
        count_2 = 0
        for d in directories:
            label_directory = os.path.join(directory, d)
            file_names = (os.path.join(label_directory, f)
                          for f in os.listdir(label_directory)
                          if f.endswith(".jpeg"))
            count_1 = count_1 + 1
            for f in file_names:
                images.append(cv2.imread(f, cv2.IMREAD_GRAYSCALE))
                if d == "NORMAL":
                    labels.append(0)
                elif d == "PNEUMONIA":
                    labels.append(1)
                count_2 = count_2 + 1
        images, labels = np.array(images), np.array(labels)
        images = np.array([cv2.resize(img, (50, 50)) for img in images])
        return images, labels

    X_train, y_train = load_data('chest_xray/train')
    X_test, y_test = load_data('chest_xray/test')
    X_val, y_val = load_data('chest_xray/val')

    '''note that as the input image is greyscale, we have to add one dimension to the array representation of the 
    image(input channel is 1) so it can be later correctly processed by pytorch '''
    # np.savez('dataset.npz', X_train=X_train[..., np.newaxis], X_test=X_test[..., np.newaxis], X_val=X_val[..., np.newaxis], y_train=y_train, y_test=y_test, y_val=y_val)
    return X_train[..., np.newaxis], X_test[..., np.newaxis], X_val[..., np.newaxis], y_train, y_test, y_val


class BasicDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, idx):
        return dict(X=self.X[idx], y=self.y[idx])

    def __len__(self):
        return self.X.shape[0]


def train_image_classifier(model, dataset, learning_rate, batch_size, epochs, device):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        data_loader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        epoch_loss = 0.0

        for batch in data_loader:
            model.zero_grad()
            model.zero_grad()

            yhat = model.forward(batch['X'].float().to(device))

            batch_loss = F.cross_entropy(yhat, batch['y'].long().to(device))
            epoch_loss += batch_loss.item()

            batch_loss.backward()
            optimizer.step()

        if (epoch == 0) or (((epoch + 1) % 10) == 0):
            print(f'Epoch {epoch + 1}/{epochs} - Loss: {epoch_loss}')

            
class ImageDataset(object):
    def __init__(self, X, y):
        self.X = np.moveaxis(X, -1, 1)
        self.y = y

    def __getitem__(self, idx):
        return dict(X=self.X[idx], y=self.y[idx])

    def __len__(self):
        return self.X.shape[0]


class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # You can figure out this number by printing the shape of the previous layer.
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 62)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.shape) to figure out the size of your linear layer
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [2]:
# Load training and testing data.
'''You can uncomment the line of code below to read the data directly, but it is not recommended as it will take a 
very long time, see the function load_dataset() to check how the original images are resized and saved as numpy array'''
# X_train, X_test, X_val, y_train, y_test, y_val = load_dataset()

dataset = np.load('dataset.npz')
X_train, X_test, X_val, y_train, y_test, y_val = dataset['X_train'], dataset['X_test'], dataset['X_val'], \
                                                 dataset['y_train'], dataset['y_test'], dataset['y_val']
print('Training samples:', X_train.shape[0])
print('Testing samples:', X_test.shape[0])
print('Image shape:', X_train[0].shape)
print('#Classes:', len(np.unique(y_train)))

Training samples: 5216
Testing samples: 624
Image shape: (50, 50, 1)
#Classes: 2


In [3]:
# Training
device = torch.device('cuda')
cnn = CNN().to(device)
dataset = ImageDataset(X_train, y_train)
learning_rate = 0.005
batch_size = 256
epochs = 100
train_image_classifier(cnn, dataset, learning_rate, batch_size, epochs, device)

d:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 1/100 - Loss: 51.454414546489716
Epoch 10/100 - Loss: 1.0653930380940437
Epoch 20/100 - Loss: 0.3174277297803201
Epoch 30/100 - Loss: 0.5307612037286162
Epoch 40/100 - Loss: 0.33054596494184807
Epoch 50/100 - Loss: 0.1766279524890706
Epoch 60/100 - Loss: 0.0005869783999514766
Epoch 70/100 - Loss: 0.00032407975322712446
Epoch 80/100 - Loss: 0.00020893043540581857
Epoch 90/100 - Loss: 0.00014561474381480366
Epoch 100/100 - Loss: 0.00010678852504497627


In [4]:
# Evaluation
from sklearn.metrics import precision_recall_fscore_support

dataset_test = ImageDataset(X_test, y_test)

with torch.no_grad():
    X = torch.from_numpy(np.array([sample['X'] for sample in dataset_test])).float().to(device)
    yhat_unnormalized = cnn.forward(X).detach().cpu().numpy()

yhat = np.argmax(yhat_unnormalized, axis=1)
y = np.array([sample['y'] for sample in dataset_test])
prec, rec, f1, _ = precision_recall_fscore_support(y, yhat, average='weighted')

print(f'Precision: {prec:.03}')
print(f'Recall: {rec:.03}')
print(f'F1-score: {f1:.03}')

Precision: 0.805
Recall: 0.742
F1-score: 0.699


In [6]:
# training and evaluation with data augmentation
'''above I have done the training and evaluation without data augmentation, below, I will repeat the process again 
but with data augmentation '''

print(np.unique(y_train, return_counts=True))
'''note the printed result above and with some simple testing, it is not difficult to see that the first 1341 samples are 
normal and the rest 3875 are pneumonia, my method of data augmentation is doing undersampling of the majority class'''
length_of_minority_class = 1341
X_pneumonia = X_train[length_of_minority_class:]
X_pneumonia_undersampled = np.zeros((length_of_minority_class, 50, 50, 1), dtype=float)
y_pneumonia_undersampled = np.zeros(length_of_minority_class, dtype=int)
import random
index_list = sorted(random.sample(range(3875), length_of_minority_class))

for i in range(length_of_minority_class):
    X_pneumonia_undersampled[i] = X_pneumonia[index_list[i]]
    y_pneumonia_undersampled[i] = 1

X_train_augmented = np.concatenate((X_train[:length_of_minority_class], X_pneumonia_undersampled), axis=0)
y_train_augmented = np.concatenate((y_train[:length_of_minority_class], y_pneumonia_undersampled), axis=0)
print('Training samples:', X_train_augmented.shape[0])
print('Testing samples:', X_test.shape[0])
print('Image shape:', X_train_augmented[0].shape)
print('#Classes:', len(np.unique(y_train_augmented)))

(array([0, 1]), array([1341, 3875], dtype=int64))
Training samples: 2682
Testing samples: 624
Image shape: (50, 50, 1)
#Classes: 2


In [7]:
# Training
cnn = CNN().to(device)
dataset = ImageDataset(X_train_augmented, y_train_augmented)
learning_rate = 0.005
batch_size = 256
epochs = 100
train_image_classifier(cnn, dataset, learning_rate, batch_size, epochs, device)

Epoch 1/100 - Loss: 81.74185240268707
Epoch 10/100 - Loss: 3.217377334833145
Epoch 20/100 - Loss: 1.962774708867073
Epoch 30/100 - Loss: 1.6188807860016823
Epoch 40/100 - Loss: 1.1732525676488876
Epoch 50/100 - Loss: 0.8777704685926437
Epoch 60/100 - Loss: 0.8536972030997276
Epoch 70/100 - Loss: 0.6338348351418972
Epoch 80/100 - Loss: 0.5879975091665983
Epoch 90/100 - Loss: 0.3624602062627673
Epoch 100/100 - Loss: 0.33988139778375626


In [8]:
# Evaluation
from sklearn.metrics import precision_recall_fscore_support

dataset_test = ImageDataset(X_test, y_test)

with torch.no_grad():
    X = torch.from_numpy(np.array([sample['X'] for sample in dataset_test])).float().to(device)
    yhat_unnormalized = cnn.forward(X).detach().cpu().numpy()

yhat = np.argmax(yhat_unnormalized, axis=1)
y = np.array([sample['y'] for sample in dataset_test])
prec, rec, f1, _ = precision_recall_fscore_support(y, yhat, average='weighted')

print(f'Precision: {prec:.03}')
print(f'Recall: {rec:.03}')
print(f'F1-score: {f1:.03}')

Precision: 0.832
Recall: 0.816
F1-score: 0.804
